##### Copyright 2019 The TensorFlow Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Memuat sebuah pandas.DataFrame

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/load_data/pandas_dataframe"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Lihat di TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/load_data/pandas_dataframe.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Jalankan di Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/load_data/pandas_dataframe.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Lihat source di GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/load_data/pandas_dataframe.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Unduh notebook</a>
  </td>
</table>

Tutorial ini menunjukkan sebuah contoh bagaimana memuat dataframe pandas menjadi sebuah `tf.data.Dataset`

Tutorial ini menggunakan [dataset](https://archive.ics.uci.edu/ml/datasets/heart+Disease) kecil yang disediakan oleh Cleveland Clinic Foundation for Heart Disease. Terdapat beberapa ratus baris data dalam CSV. Setiap baris mendeskripsikan seorang pasien, dan setiap kolom menjelaskan atribut dari pasien tersebut. Kita akan menggunakan informasi ini untuk memprediksi apakah seorang pasien memiliki penyakit jantung, dimana dalam dataset ini merupakan permasalahan klasifikasi biner.

## Membaca data menggunakan pandas

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version hanya terdapat di Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import pandas as pd
import tensorflow as tf

Unduh file csv yang berisi data jantung pasien.

In [0]:
csv_file = tf.keras.utils.get_file('heart.csv', 'https://storage.googleapis.com/applied-dl/heart.csv')

Baca csv file menggunakan pandas.

In [0]:
df = pd.read_csv(csv_file)

In [0]:
df.head()

In [0]:
df.dtypes

Ubah kolom `thal` yang merupakan sebuah data bertipe `object` pada dataframe menjadi tipe data numerik diskrit.

In [0]:
df['thal'] = pd.Categorical(df['thal'])
df['thal'] = df.thal.cat.codes

In [0]:
df.head()

## Muat data menggunakan `tf.data.Dataset`

Gunakan `tf.data.Dataset.from_tensor_slices` untuk membaca nilai dari dataframe pandas.

Salah satu keunggulan menggunakan `tf.data.Dataset` adalah Anda dapat menuliskan pipeline data yang sederhana dan sangat efisien. Baca [Petunjuk memuat data](https://www.tensorflow.org/guide/data) untuk mengetahui lebih banyak lagi tentang hal ini.

In [0]:
target = df.pop('target')

In [0]:
dataset = tf.data.Dataset.from_tensor_slices((df.values, target.values))

In [0]:
for feat, targ in dataset.take(5):
  print ('Features: {}, Target: {}'.format(feat, targ))

Karena `pd.Series` mengimplementasikan protokol `__array__`, hal ini menyebabkan Anda dapat menggunakan `np.array` atau `tf.Tensor`.

In [0]:
tf.constant(df['thal'])

Lakukan shuffle dan batch dataset.

In [0]:
train_dataset = dataset.shuffle(len(df)).batch(1)

## Buat dan latih sebuah model

In [0]:
def get_compiled_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
  ])

  model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])
  return model

In [0]:
model = get_compiled_model()
model.fit(train_dataset, epochs=15)

## Alternatif dari *feature columns*

Menggunakan sebuah *dictionary* sebagai input untuk sebuah model sama mudahnya dengan membuat *matching dictionary* dari layer `tf.keras.layers.Input`, melakukan setiap proses pre-processing dan kemudian melakukan *stacking* terhadap layer tersebut menggunakan [api fungsional](../../guide/keras/functional.ipynb). Anda dapat menggunakan ini sebagai alternatif dari [*feature columns*](../keras/feature_columns.ipynb).

In [0]:
inputs = {key: tf.keras.layers.Input(shape=(), name=key) for key in df.keys()}
x = tf.stack(list(inputs.values()), axis=-1)

x = tf.keras.layers.Dense(10, activation='relu')(x)
output = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model_func = tf.keras.Model(inputs=inputs, outputs=output)

model_func.compile(optimizer='adam',
                   loss='binary_crossentropy',
                   metrics=['accuracy'])

Cara termudah untuk menjaga struktur kolom dari `pd.DataFrame` ketika digunakan dengan `tf.data` adalah dengan cara mengkonversi `pd.DataFrame` menjadi sebuah `dict`, kemudian melakukan *slicing* dari *dictionary* tersebut.

In [0]:
dict_slices = tf.data.Dataset.from_tensor_slices((df.to_dict('list'), target.values)).batch(16)

In [0]:
for dict_slice in dict_slices.take(1):
  print (dict_slice)

In [0]:
model_func.fit(dict_slices, epochs=15)